In [1]:
from konlpy.tag import Mecab
mecab = Mecab('c:/mecab/mecab-ko-dic')
morph = []
with open('./wiki_preprocess_sample_normalize_light_no_enter.txt', encoding='utf-8') as file:
    '''
        bert 권장 wordpiece 방식인 'bpe는 띄어쓰기 기준으로 문장을 어절 단위로 최소단위로 쪼개서' 구분하여 빈도를 파악하므로,
        ex) 아버지가 방에 들어가신다. -> 아버지가/방에/들어가신다. -> 아/#버/#지/#가/#방/#에/#들/#어/#가/#신/#다/#.
        순서만 살짝 바꿔 Mecab을 적용해보았다. (먼저 어절을 쪼개주고 띄어쓰기로 구분하여, bpe에서 빈도를 계산하도록)
        ex) 아버지가 방에 들어가신다. -> 아버지/가/방/에/들어가/신다/. -> 아버지/#가/#방/#에/#들어가/#신다/#.
        
    '''
    morphs_sents = list(map(lambda x:' '.join(mecab.morphs(x.replace('\n',''))), file.readlines()))
# mecab.pos(sentence)
# copus_list = mecab.morphs(sentence)

In [2]:
print(morphs_sents[:10])

['제임스 얼 " 지미 " 카터 주니어 , 1924 년 10 월 1 일 는 민주당 출신 미국 39 대 대통령 1977 년 1981 년 이 다 .', '지미 카터 는 조지 아주 섬터 카운티 플 레인스 마을 에서 태어났 다 .', '조지아 공과 대학교 를 졸업 하 였 다 .', '그 후 해군 에 들어가 전함 원자력 잠수함 의 승무원 으로 일 하 였 다 .', '1953 년 미국 해군 대위 로 예편 하 였 고 이후 땅콩 면화 등 을 가꿔 많 은 돈 을 벌 었 다 .', '그 의 별명 이 " 땅콩 농부 " Peanut Farmer 로 알려졌 다 .', '1962 년 조지아 주 상원 의원 선거 에서 낙선 하 나 그 선거 가 부정 선거 였음을 입증 하 게 되 어 당선 되 고 , 1966 년 조지아 주 지사 선거 에 낙선 하 지만 1970 년 조지아 주 지사 를 역임 했 다 .', '대통령 이 되 기 전 조지 아주 상원 의원 을 두 번 연임 했으며 , 1971 년 부터 1975 년 까지 조지 아 지사 로 근무 했 다 .', '조지아 주지사 로 지내 면서 , 미국 에 사 는 흑인 등 용법 을 내세웠 다 .', '1976 년 미합중국 미국 제 39 대 대통령 선거 에 민주당 후보 로 출마 하 여 도덕주의 정책 으로 내세워서 , 많 은 지지 를 받 고 제럴드 포드 당시 미국 대통령 를 누르 고 당선 되 었 다 .']


In [3]:
from collections import Counter
from collections import defaultdict
import argparse

test = False

if test:
    data=[]
    with open('./wiki_20190620_small.txt', 'r', encoding="utf-8") as raw:
        for r in raw:
            if r !='\n':
                data.append(r[:-1])
    # Mecab에서 띄어쓰기로 쪼개놓았으니, 해당 문장을 스플릿하여 어절이라고 생각하고, 카운트를 계산한다.
    vocabs = Counter((eojeol.replace('_', '') for sent in data for eojeol in sent.split() if eojeol))
    vocabs = {' ##'.join(w) : c for w,c in vocabs.items() if w}
    print(vocabs)

if not test:
    #Mecab에서 띄어쓰기로 쪼개놓았으니, 해당 문장을 스플릿하여 어절이라고 생각하고, 카운트를 계산한다.
    vocabs = Counter((eojeol.replace('_', '') for sent in morphs_sents for eojeol in sent.split() if eojeol))
    with open('./vocabs_count.txt', 'w', encoding="utf-8") as raw:
        # 쪼개기가 너무 커 파일로 꺼내서 출력 확인
        raw.write(str(vocabs))
    vocabs = {' ##'.join(w) : c for w,c in vocabs.items() if w}
    with open('./vocabs_insert_sharp.txt', 'w', encoding="utf-8") as raw:
        # 쪼개기가 너무 커 파일로 꺼내서 출력 확인
        raw.write(str(vocabs))

In [4]:
n_iters = 5000
verbose=True
def get_stats(vocabs):
    pairs = defaultdict(int)
    for word, freq in vocabs.items():
#         print("maked word pair : {}, {}".format(word, freq))
        symbols = word.split()
        for i in range(len(symbols)-1):
            # 한글자는 진행하지 않으며, pairs에 빈 tuple이 리턴됨.
            pairs[(symbols[i],symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = ' '.join(pair)
    back_pair = pair[1]
    replacer = pair[0] +  back_pair.replace("##","")
    for word, freq in v_in.items():
        w_out = word.replace(bigram, replacer)
        v_out[w_out] = freq
    return v_out

# 한 글자씩의 char의 서순별 등장횟수를 찾아서, 한 단어의 word로 만드는 작업을 n_iters 만큼 한다.
# n_iters가 10이라면, 최대 등장할 수 있는 max word length는 10을 넘을 수 없다.______의견확인필요
for i in range(n_iters + 1):
    pairs = get_stats(vocabs)    # 등장 단어별 각 캐릭터 시퀀스별로 쪼개기.
#     print("finded word pair : ", pairs)
    if not pairs:
        print('not pair appear')
        break
    best = max(pairs, key=pairs.get)    # 가장 많이 등장하는 pair 벨류를 찾음
#     print(i,best)
    vocabs = merge_vocab(best, vocabs)    # 가장 많이 등장하는 pair 벨류를 합쳐서 세트로 만듬
    if verbose and i % 100 == 99:
        print('\rtraining bpe {} / {}'.format(i+1, n_iters), end='\n', flush=True)
if verbose:
    print('\rtraining bpe was done{}'.format(' '*40))

units = {}
for word, freq in vocabs.items():
    for unit in word.split():
        units[unit] = units.get(unit, 0) + freq
max_length = max((len(w) for w in units))

print(max_length)

training bpe 100 / 10000
training bpe 200 / 10000
training bpe 300 / 10000
training bpe 400 / 10000
training bpe 500 / 10000
training bpe 600 / 10000
training bpe 700 / 10000
training bpe 800 / 10000
training bpe 900 / 10000
training bpe 1000 / 10000
training bpe 1100 / 10000
training bpe 1200 / 10000
training bpe 1300 / 10000
training bpe 1400 / 10000
training bpe 1500 / 10000
training bpe 1600 / 10000
training bpe 1700 / 10000
training bpe 1800 / 10000
training bpe 1900 / 10000
training bpe 2000 / 10000
training bpe 2100 / 10000
training bpe 2200 / 10000
training bpe 2300 / 10000
training bpe 2400 / 10000
training bpe 2500 / 10000
training bpe 2600 / 10000
training bpe 2700 / 10000
training bpe 2800 / 10000
training bpe 2900 / 10000
training bpe 3000 / 10000
training bpe 3100 / 10000
training bpe 3200 / 10000
training bpe 3300 / 10000
training bpe 3400 / 10000
training bpe 3500 / 10000
training bpe 3600 / 10000
training bpe 3700 / 10000
training bpe 3800 / 10000
training bpe 3900 / 1

In [5]:
for unit, frequency in sorted(units.items(), key=lambda x:(-x[1], -len(x[0]))):
    print(unit,frequency)

. 1309567
의 1103373
다 1047806
이 1020314
는 1001286
, 954815
에 851090
을 789014
하 743123
은 580466
를 476444
년 443478
고 421316
있 359328
가 356618
에서 356474
으로 351862
로 319431
되 318862
한 296960
었 291039
과 226578
들 211070
월 195655
와 186853
도 186306
했 176934
였 176567
일 176303
적 171328
인 160909
그 160856
기 146430
어 143079
것 122463
" 120750
여 120253
지 119784
1 112943
게 111822
수 105502
2 104050
한다 102760
등 95125
며 91527
' 87284
된 84759
3 81823
해 73216
던 71571
할 66743
자 65902
만 64166
4 62231
대 61495
아 61229
명 61005
중 59964
주 58891
않 58411
았 57311
까지 56614
제 55570
5 55270
으며 54561
개 53043
성 52998
에게 52950
받 51369
후 50878
시 49986
지만 48620
세 47580
부터 46524
면 46348
나 46252
된다 46106
전 46091
때 44747
6 44445
같 43597
라고 43400
라는 42045
10 41718
이후 41375
번 40608
8 40391
면서 40340
7 40232
사용 40204
말 39796
두 39147
없 39098
라 38579
는데 37663
일본 36447
미국 35921
사 34533
위 34529
화 34388
군 34267
9 34128
지역 31959
##스 31704
##자 31546
##지 31307
오 31286
서 31202
때문 31164
대한 31010
12 31004
다고 30349
보 30342
##기 29856
및 29541
다

제한 3105
알려진 3103
농업 3101
##ad 3098
병원 3098
머 3097
구단 3094
##탈 3093
##C 3093
길이 3092
값 3090
헌 3088
슬 3087
##출 3085
왕조 3083
식민지 3082
##턴 3080
오래 3080
챔피언 3076
이듬해 3073
록 3073
통하 3072
1945 3071
통신 3067
해결 3066
유사 3065
가정 3064
##매 3057
##골 3057
기여 3055
아서 3054
로써 3054
기초 3051
법률 3044
r 3041
휘 3038
기념 3034
Y 3032
결성 3031
중간 3030
기계 3029
배치 3027
제안 3024
영역 3018
바다 3017
##필 3013
흑 3010
연대 3009
마리 3009
후반 3006
밀도 3005
지대 3002
잇 3002
연맹 3001
홀 2999
##층 2992
지도자 2987
KBS 2982
방어 2981
", 2980
전차 2979
##등 2978
만화 2978
아르 2977
올랐 2976
성립 2974
권력 2974
내각 2974
먼저 2974
아일랜드 2972
##갈 2971
1950 2970
용어 2970
의식 2968
빌 2968
콘 2966
##X 2964
코치 2963
지적 2961
베르 2961
##울 2958
협력 2955
서울특별시 2953
폭 2950
인사 2948
사진 2948
##ion 2947
민주당 2947
숙 2947
신문 2936
나머지 2931
이하 2930
승격 2930
출판 2925
그대로 2924
##좌 2922
품 2920
가리킨다 2917
##나무 2916
##쳐 2914
츠 2912
함수 2909
등록 2909
당했 2909
타이틀 2907
특수 2905
##립 2904
##범 2904
스웨덴 2900
##불 2895
경기장 2894
공장 2894
마찬가지 2893
만큼 2891
장소 2891
타이 2889
특 2888
근처 2887
##규 2886
범 2884
벤 2881
괴 

보관 1204
숭 1204
44 1203
방정식 1202
세이 1202
국기 1202
주력 1202
사령부 1201
##혼 1201
혼합 1201
성분 1201
##리스 1200
히틀러 1200
질병 1200
확률 1200
정하 1200
통산 1200
데이 1199
날개 1199
바이러스 1198
임의 1198
재배 1198
향해 1198
전면 1198
저작 1198
##폭 1197
##틱 1197
완료 1197
당국 1197
양쪽 1197
추천 1196
##ue 1195
스튜디오 1195
라이트 1195
토론 1194
꼬리 1194
세르비아 1193
된다고 1193
다며 1193
블루 1193
홍보 1193
동서 1192
세워 1191
스탠 1191
하천 1191
##뇌 1190
서구 1190
청구 1190
to 1190
싸 1190
그리하여 1189
연구원 1189
현역 1189
크로아티아 1188
##팀 1188
화재 1188
대량 1187
람 1186
대수 1185
가공 1185
민중 1185
##없이 1184
프리미어 1184
텍사스 1184
경로 1184
동방 1184
##웨이 1183
지사 1183
가담 1183
욕 1183
u 1183
가량 1182
훈장 1182
공정 1182
페르 1181
동해 1181
분위기 1180
묶 1180
춤 1180
##렐 1179
아이돌 1179
마이 1179
황금 1179
애플 1179
##저우 1178
폭탄 1178
신뢰 1178
콘서트 1177
##럴 1177
랜 1177
한정 1176
54 1176
멀리 1176
총회 1176
기기 1176
섞 1176
사운드 1175
위상 1175
추방 1175
속해 1175
한글 1175
막부 1174
택 1174
##ill 1173
느끼 1173
불만 1173
아웃 1172
병합 1170
서버 1170
전형 1169
프라 1169
농촌 1168
사퇴 1167
생긴 1166
##탕 1165
레코드 1164
듯이 1164
분해 1164
레오 1162
파르 1162
순간 1

모집 647
해전 647
키스 647
##피아 646
필립 646
84 646
스쿨 646
서울시 645
낸다 645
버튼 645
홍수 645
빠 645
독립운동가 644
##데르 644
##aw 644
하워드 644
덧붙여 644
유해 644
민회 644
남아메리카 643
##코스 643
소니 643
말라 643
소총 643
꼴 643
##미터 642
Love 642
부동산 642
노드 642
97 642
조용 642
제트 642
곳곳 642
##ung 641
아울러 641
##컨 641
경부 641
리버 641
아바 641
흙 641
쇄 641
67 640
가득 640
닥터 640
고립 640
회수 640
식민 640
자고 640
스토 640
소금 640
나리 640
얀 640
곁 640
##one 639
귀환 639
고무 639
슬로베니아 638
그럼에도 638
프랑스군 638
##하이 638
동영상 638
역전 638
압박 638
회계 638
민족주의 637
엄마 637
알라 637
재즈 637
수명 637
74 637
이종 637
둥 637
면허 636
71 636
무인 636
승점 636
닿 636
` 636
못한다 635
160 635
이명박 635
동메달 635
이정 635
권고 635
환원 635
자율 635
드림 635
최신 635
첼시 635
##철도 634
피렌체 634
도리 634
소환 634
사사 634
폭격 634
물리학자 633
주권 633
기병 633
언제 633
부자 633
노인 633
염증 633
남극 633
쾌속 633
##히로 632
스케이팅 632
or 632
동상 632
##ary 631
미얀마 631
착공 631
풀려 631
09 631
용인 631
공자 631
##바이 630
##숭 630
##첼 630
순조 630
못해 630
전장 630
정전 630
추증 629
색상 629
주어진 628
벌어졌 628
김영 628
기금 628
제곱 628
미즈 628
추격 628
모노 628
칼뱅 628
신하 628
잊 628


담겨 445
교양 445
산란 445
연봉 445
트랜 445
순회 445
NC 445
스콧 445
##net 444
##슈타인 444
##에르 444
##슬러 444
경쟁력 444
소나무 444
더욱이 444
Gro 444
붙여진 444
111 444
언어학 444
삼각 444
누가 444
동국 444
Ag 444
게르 444
지옥 444
##다운 443
방사선 443
주어지 443
부회장 443
나올 443
기다 443
CH 443
눌 443
##int 442
합스부르크 442
##무스 442
승부차기 442
비로소 442
Sch 442
누군가 442
느껴 442
둘레 442
강희 442
김태 442
가까워 441
폭넓 441
도라 441
공습 441
상공 441
어로 441
바이올린 440
##트리 440
##랴 440
섭정 440
속편 440
개종 440
갱신 440
자선 440
유라 440
앗 440
##가루 439
평정 439
당주 439
체력 439
오노 439
모자 439
로프 439
단행 439
크롬 439
##학자 438
##다니 438
도카이도 438
##곽 438
Tra 438
박람회 438
소피아 438
미스터 438
부재 438
연이 438
각도 438
채소 438
피스 438
에디 438
케임브리지 437
##타고 437
##IT 437
엔지니어 437
##대군 437
##뤄 437
연기자 437
이관 437
육체 437
부채 437
소매 437
담긴 437
홈즈 437
오키 437
펌 437
##스케 436
##교육 436
##뤘 436
성취 436
스핀 436
실망 436
이원 436
사육 436
SN 436
이남 436
아데 436
브라이언 435
두개골 435
입체 435
동아 435
장미 435
용암 435
헝 435
##ome 434
##TV 434
서유럽 434
우승자 434
##쌍 434
통치자 434
705 434
기하 434
연수 434
신조 434
곡인 434
컨셉 434
서민 434
Ap 434
앱 434
##o

...

##어서부터 1
##클로스키 1
##psel 1
##ctod 1
Wheres 1
Trades 1
##kege 1
186076 1
##INAS 1
Tounen 1
##테리아기 1
Accele 1
##nene 1
##irly 1
##폴리투스 1
##ylot 1
258053 1
102800 1
114100 1
164000 1
168000 1
313200 1
Mander 1
##OBOS 1
Crashe 1
##미트레스 1
144000 1
rement 1
Partou 1
##ikif 1
##ISDC 1
oricon 1
##edec 1
##icec 1
berser 1
##dech 1
comune 1
##eary 1
429598 1
Planec 1
Serere 1
189064 1
Lament 1
##inir 1
##iush 1
##EUSP 1
##aktr 1
Mantou 1
##ECAD 1
##더스카이 1
##arkg 1
##참여자치 1
##utoh 1
Roseng 1
##라노스케 1
##ftre 1
##ftex 1
##spry 1
##kten 1
000027 1
000006 1
Temeng 1
##IMPS 1
##AMBO 1
##이에스티 1
##aios 1
634209 1
511188 1
489531 1
156906 1
106677 1
145430 1
170574 1
Cortel 1
##itew 1
000424 1
##Nell 1
##INES 1
Gespen 1
##fted 1
##Onet 1
141592 1
Collen 1
##ietr 1
##ilel 1
Parell 1
##사라고사 1
##ctok 1
Marder 1
Latour 1
##카를로스 1
##스튜디오 1
##MCOR 1
##ITOR 1
##aset 1
##kmet 1
Lalala 1
##나리아과 1
Stares 1
##UNGM 1
##FCON 1
##auto 1
##mela 1
케임브리지만 1
##ETCO 1
Mantov 1
##SONA 1
##Tong 1
Amenem 1
##ikep 1
##issp 1
진

##GOS 1
##미노리 1
##동문헌 1
##로스타 1
플라이오세 1
##DCP 1
광주교도소 1
Sutor 1
##토대회 1
13328 1
57542 1
Angen 1
detra 1
49656 1
65746 1
comel 1
Parec 1
##NTC 1
Serev 1
##레이다 1
redem 1
Pheng 1
Hemel 1
##전산업 1
##할리어 1
존스튜어트 1
##페로니 1
##MCD 1
Abell 1
##코미디 1
06374 1
06354 1
06387 1
06332 1
노스포인트 1
spele 1
05267 1
##Rob 1
##GOL 1
LACON 1
Rotro 1
11116 1
##BSc 1
##암시성 1
##Cot 1
Toung 1
##PSc 1
##GSM 1
##342 1
10257 1
##미니아 1
##러스테 1
##pew 1
AMSAT 1
##경제학 1
99636 1
##드라크 1
Tenep 1
Helan 1
Susce 1
trong 1
##사무기 1
Pared 1
김정일리아 1
10909 1
##스트레 1
##무인격 1
Tomel 1
##타이사 1
##이지지 1
##스트루 1
Allat 1
11430 1
Itall 1
Amear 1
94573 1
##프스터 1
NEEDS 1
Levon 1
##밀라노 1
20371 1
알레이스크 1
##Jen 1
29277 1
Macer 1
Rotog 1
##메이저 1
Musev 1
23271 1
##440 1
##태어났 1
Jenev 1
Alate 1
ASOSA 1
##벨라스 1
##가리우 1
ITAKO 1
##Gur 1
Golar 1
##USM 1
##GMA 1
##마무리 1
56637 1
80635 1
노스케이프 1
10339 1
Comex 1
10176 1
Rotom 1
11066 1
22313 1
Arash 1
##DSL 1
##성전리 1
stram 1
ADCOM 1
98756 1
98751 1
98466 1
14700 1
##카이와 1
Arang 1
##TOC 1
##GOC 1
##Sug 1


6427 1
7111 1
6724 1
6628 1
KOSE 1
2487 1
DSSC 1
lana 1
Mere 1
8592 1
옮긴다면 1
2991 1
2992 1
Nort 1
9632 1
4750 1
5340 1
byla 1
8544 1
##47 1
9856 1
Sere 1
운수소관 1
Subl 1
3963 1
당연상인 1
##59 1
##본적 1
AKOM 1
PCEC 1
Sede 1
0181 1
8128 1
8127 1
8129 1
##마녀 1
##재편 1
0549 1
0675 1
0676 1
0678 1
0679 1
0677 1
유지인사 1
##가문 1
BMAC 1
0351 1
6425 1
7180 1
8631 1
8630 1
8899 1
8611 1
8602 1
8045 1
8615 1
8049 1
날아내리 1
##자는 1
4311 1
8546 1
6006 1
Romu 1
Remu 1
##느님 1
친선고속 1
2496 1
6121 1
9330 1
3765 1
3518 1
2854 1
3132 1
비교전국 1
5833 1
atsk 1
7035 1
7017 1
7061 1
루시어스 1
0512 1
베이시에 1
5785 1
##반란 1
##잡혀 1
5579 1
##현은 1
2791 1
9862 1
8751 1
7290 1
7466 1
쓴다고도 1
대상경주 1
7592 1
4326 1
8683 1
벌인다면 1
4766 1
5710 1
6899 1
##이끌 1
8062 1
3706 1
##경유 1
2591 1
##종료 1
SADF 1
2913 1
0147 1
##무릎 1
##실무 1
##반역 1
Quem 1
spem 1
Gere 1
KIAS 1
4416 1
4499 1
엔드리스 1
아리모토 1
3926 1
Deme 1
unev 1
8232 1
5769 1
8037 1
4591 1
5771 1
8361 1
콜라이트 1
8058 1
3499 1
3582 1
GMAC 1
6086 1
7628 1
7233 1
8443 1
4778 1
8197 1
5889 1
##일부 1

0368 1
0346 1
0282 1
0356 1
0462 1
0414 1
0276 1
0432 1
0354 1
0372 1
0458 1
0338 1
0318 1
0488 1
0378 1
0478 1
0476 1
0370 1
0348 1
0328 1
0380 1
Mamu 1
alex 1
5215 1
5216 1
안드레지 1
##전부 1
NECA 1
2986 1
5004 1
7183 1
7842 1
8490 1
5241 1
lash 1
3454 1
trar 1
4116 1
##군정 1
2315 1
4058 1
ASFA 1
3470 1
FISM 1
벨라비아 1
7489 1
9055 1
오리아나 1
UNEC 1
0460 1
개선사지 1
FAMA 1
7166 1
4135 1
속한다면 1
4808 1
3647 1
##치르 1
##절망 1
##서위 1
0614 1
IISE 1
##참가 1
Lell 1
프레지어 1
##청춘 1
2506 1
마이하마 1
8975 1
##이글 1
seme 1
##감마 1
5279 1
6108 1
5719 1
Labr 1
DCOM 1
##점령 1
9949 1
구축함대 1
7623 1
THEN 1
9430 1
6262 1
6313 1
마리버드 1
##미루 1
마스페로 1
Smet 1
트라이치 1
방사성원 1
토시노리 1
0046 1
9032 1
엘리스타 1
9674 1
9562 1
5458 1
3422 1
4027 1
2770 1
ISCA 1
오스미리 1
2447 1
7630 1
칼리포르 1
kton 1
제이세라 1
한국기계 1
성남시민 1
크라마리 1
Oram 1
##모하 1
6307 1
9803 1
루이지도 1
laue 1
9284 1
8016 1
4567 1
Elat 1
아이토르 1
이시아르 1
5774 1
4112 1
3959 1
8437 1
3326 1
고교토론 1
2477 1
3043 1
노스리틀 1
8536 1
2551 1
추정신탁 1
IMPP 1
9203 1
7255 1
3180 1
9033 1
9134 1
9034 1
5505 1

매리얼 1
비단어 1
세네트 1
아세리 1
독성도 1
에이시 1
김성경 1
대부산 1
제이센 1
저장기 1
FMV 1
이상업 1
라마트 1
주전도 1
GNH 1
사카나 1
##녈 1
스크레 1
연기회 1
거처지 1
레이종 1
시간다 1
이해진 1
##꺄 1
변동원 1
성주기 1
유로스 1
베이치 1
부장기 1
개방법 1
김수령 1
하리드 1
알라크 1
지장시 1
양성우 1
현대화 1
무사상 1
무신념 1
장인원 1
스리아 1
체르크 1
브레지 1
이민주 1
배정식 1
배상호 1
호소식 1
주파나 1
전진수 1
의학생 1
토요코 1
원정일 1
그리프 1
상관장 1
김성향 1
유일자 1
강경남 1
해서정 1
공정자 1
정기동 1
오세이 1
조정치 1
최영광 1
마이노 1
##짹 1
박해인 1
정상기 1
배정국 1
신용산 1
밀어두 1
군사망 1
HDS 1
아우아 1
##ㅅ 1
##ㅌ 1
이성운 1
신민법 1
리베이 1
일상관 1
좌우지 1
오페르 1
서기준 1
칼라파 1
시티켓 1
부대기 1
발라이 1
선도사 1
미시모 1
파라기 1
조개관 1
교대성 1
극심한 1
선도르 1
이기재 1
이지휘 1
선전술 1
신호리 1
코시코 1
유리치 1
청하천 1
조기간 1
플레일 1
정지로 1
할리나 1
공사우 1
동남극 1
가정과 1
다리우 1
LAC 1
올라도 1
화성화 1
이제정 1
##숌 1
도산지 1
세라이 1
tof 1
안정책 1
당시기 1
세이자 1
양상용 1
헤르트 1
반지주 1
이명제 1
디오카 1
치카라 1
임정규 1
다테오 1
시장기 1
성문장 1
다니스 1
##귓 1
계곡선 1
바이외 1
시바이 1
자오류 1
유연동 1
도모리 1
무수정 1
장인근 1
해남도 1
진전리 1
나카바 1
가정리 1
지내리 1
오타르 1
오세라 1
비유전 1
절대치 1
전문화 1
배상하 1
유성도 1
sew 1
안정한 1
주변화 1
가마스 1
임기리 1
이나중 1
GNC 1
로스포 1
오스타 1
가이자 1
종자리 1
신경성 1
폐기름 1
니시아 1
배수정 1
여운영 1
강연재 

In [6]:
with open('./sample_normalize_light_no_enter_preproc.txt', 'w', encoding='utf-8') as f:
    f.write('[CLS]\n[UNK]\n[SEP]\n[PAD]\n[MASK]\n')
    # 저장은 등장빈도가 많은 문장부터 저장되게 된다.
    for unit, frequency in sorted(units.items(), key=lambda x:(-x[1], -len(x[0]))):
        f.write('{}\n'.format(unit))

In [7]:
import pickle

f = open("wiki_preprocess_sample_normalize_light_no_enter.pkl","wb")
pickle.dump(sorted(units.items(), key=lambda x:(-x[1], -len(x[0]))),f)
f.close()

In [8]:
import pickle

with open("wiki_preprocess_sample_normalize_light_no_enter.pkl", 'rb') as f:
    data = pickle.load(f)
data

[('.', 1309567),
 ('의', 1103373),
 ('다', 1047806),
 ('이', 1020314),
 ('는', 1001286),
 (',', 954815),
 ('에', 851090),
 ('을', 789014),
 ('하', 743123),
 ('은', 580466),
 ('를', 476444),
 ('년', 443478),
 ('고', 421316),
 ('있', 359328),
 ('가', 356618),
 ('에서', 356474),
 ('으로', 351862),
 ('로', 319431),
 ('되', 318862),
 ('한', 296960),
 ('었', 291039),
 ('과', 226578),
 ('들', 211070),
 ('월', 195655),
 ('와', 186853),
 ('도', 186306),
 ('했', 176934),
 ('였', 176567),
 ('일', 176303),
 ('적', 171328),
 ('인', 160909),
 ('그', 160856),
 ('기', 146430),
 ('어', 143079),
 ('것', 122463),
 ('"', 120750),
 ('여', 120253),
 ('지', 119784),
 ('1', 112943),
 ('게', 111822),
 ('수', 105502),
 ('2', 104050),
 ('한다', 102760),
 ('등', 95125),
 ('며', 91527),
 ("'", 87284),
 ('된', 84759),
 ('3', 81823),
 ('해', 73216),
 ('던', 71571),
 ('할', 66743),
 ('자', 65902),
 ('만', 64166),
 ('4', 62231),
 ('대', 61495),
 ('아', 61229),
 ('명', 61005),
 ('중', 59964),
 ('주', 58891),
 ('않', 58411),
 ('았', 57311),
 ('까지', 56614),
 ('제', 55570),
 ('